In [1]:
# # Importing Packages
import os
import copy
import config
import dota2api
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score

%matplotlib inline

In [2]:
# # STEAM Client API Key to access the dota2 api
API_KEY = config.STEAM_API_KEY
api = dota2api.Initialise(API_KEY)

In [3]:
data = pd.read_csv('../data/draft/capmodedata.csv')

In [4]:
data.head(3)

,match_id,radiant_win,duration,1is_pick,1team_id,1hero_id,2is_pick,2team_id,2hero_id,3is_pick,...,17hero_id,18is_pick,18team_id,18hero_id,19is_pick,19team_id,19hero_id,20is_pick,20team_id,20hero_id
0,1928898739,True,2885,False,0,73,False,1,69,False,...,41.0,False,0.0,81.0,True,1.0,105.0,True,0.0,94.0
1,1928932285,True,1894,False,0,69,False,1,102,False,...,40.0,False,0.0,28.0,True,1.0,97.0,True,0.0,62.0
2,1928903165,True,2424,False,1,85,False,0,68,False,...,30.0,False,1.0,47.0,True,0.0,58.0,True,1.0,33.0


In [5]:
# # # Converting boolean value into number
boolean_values = 20
for i in range(boolean_values):
    pick_id = '{}is_pick'.format(i+1)
    data[pick_id] = data[pick_id].apply(lambda x: 1 if x else 0)

In [6]:
data.head()

,match_id,radiant_win,duration,1is_pick,1team_id,1hero_id,2is_pick,2team_id,2hero_id,3is_pick,...,17hero_id,18is_pick,18team_id,18hero_id,19is_pick,19team_id,19hero_id,20is_pick,20team_id,20hero_id
0,1928898739,True,2885,0,0,73,0,1,69,0,...,41.0,0,0.0,81.0,1,1.0,105.0,1,0.0,94.0
1,1928932285,True,1894,0,0,69,0,1,102,0,...,40.0,0,0.0,28.0,1,1.0,97.0,1,0.0,62.0
2,1928903165,True,2424,0,1,85,0,0,68,0,...,30.0,0,1.0,47.0,1,0.0,58.0,1,1.0,33.0
3,1928907204,True,2588,0,1,71,0,0,73,0,...,30.0,0,1.0,68.0,1,0.0,50.0,1,1.0,21.0
4,1928865084,True,3658,0,1,12,0,0,42,0,...,93.0,0,1.0,67.0,1,0.0,72.0,1,1.0,40.0


In [7]:
# # Getting the wins of the second pick
temp = data.loc[data['1team_id'] == 0]
win1 = temp.loc[temp['radiant_win'] == False]
temp2 = data.loc[data['1team_id'] == 1]
win2 = temp2.loc[temp2['radiant_win'] == True]

In [8]:
# # Validation
win1.shape[0] + win2.shape[0] < len(data)

True

In [9]:
to_concat_dataframes = [win1, win2]
all_win_data_only = pd.concat(to_concat_dataframes)

In [10]:
all_win_data_only = all_win_data_only.sort_index()
all_win_data_only.head(3)

,match_id,radiant_win,duration,1is_pick,1team_id,1hero_id,2is_pick,2team_id,2hero_id,3is_pick,...,17hero_id,18is_pick,18team_id,18hero_id,19is_pick,19team_id,19hero_id,20is_pick,20team_id,20hero_id
2,1928903165,True,2424,0,1,85,0,0,68,0,...,30.0,0,1.0,47.0,1,0.0,58.0,1,1.0,33.0
3,1928907204,True,2588,0,1,71,0,0,73,0,...,30.0,0,1.0,68.0,1,0.0,50.0,1,1.0,21.0
4,1928865084,True,3658,0,1,12,0,0,42,0,...,93.0,0,1.0,67.0,1,0.0,72.0,1,1.0,40.0


In [11]:
# # # Getting Heroes
# # # Getting Heroes ID according to the name
heroes = api.get_heroes()

heroes_dict = {}
for i in range (len(heroes['heroes'])):
    hero_id = heroes['heroes'][i]['id']
    hero_name = heroes['heroes'][i]['localized_name']
    heroes_dict[hero_id]= hero_name

### Applying Machine Learning Model

### Model 1

In [15]:
# # Making Variables

hero_1 = all_win_data_only['1hero_id']
heropick_1 = all_win_data_only['1is_pick']
hero_2 = all_win_data_only['2hero_id']
heropick_2 = all_win_data_only['2is_pick']
hero_3 = all_win_data_only['3hero_id']
heropick_3 = all_win_data_only['3is_pick']
hero_4 = all_win_data_only['4hero_id']
heropick_4 = all_win_data_only['4is_pick']
hero_5 = all_win_data_only['5hero_id']
heropick_5 = all_win_data_only['5is_pick']

hero_6 = all_win_data_only['6hero_id']
heropick_6 = all_win_data_only['6is_pick']
hero_7 = all_win_data_only['7hero_id']
heropick_7 = all_win_data_only['7is_pick']
hero_8 = all_win_data_only['8hero_id']
heropick_8 = all_win_data_only['8is_pick']
hero_9 = all_win_data_only['9hero_id']
heropick_9 = all_win_data_only['9is_pick']
hero_10 = all_win_data_only['10hero_id']
heropick_10 = all_win_data_only['10is_pick']

hero_11 = all_win_data_only['11hero_id']
heropick_11 = all_win_data_only['11is_pick']
hero_12 = all_win_data_only['12hero_id']
heropick_12 = all_win_data_only['12is_pick']
hero_13 = all_win_data_only['13hero_id']
heropick_13 = all_win_data_only['13is_pick']
hero_14 = all_win_data_only['14hero_id']
heropick_14 = all_win_data_only['14is_pick']
hero_15 = all_win_data_only['15hero_id']
heropick_15 = all_win_data_only['15is_pick']

hero_16 = all_win_data_only['16hero_id']
heropick_16 = all_win_data_only['16is_pick']
hero_17 = all_win_data_only['17hero_id']
heropick_17 = all_win_data_only['17is_pick']
hero_18 = all_win_data_only['18hero_id']
heropick_18 = all_win_data_only['18is_pick']
hero_19 = all_win_data_only['19hero_id']
heropick_19 = all_win_data_only['19is_pick']
hero_20 = all_win_data_only['20hero_id']
heropick_20 = all_win_data_only['20is_pick']

In [16]:
test1 = pd.DataFrame(columns=['1hero_id','1is_pick' ,'2hero_id','2is_pick', '3hero_id','3is_pick', '4hero_id', '4is_pick', '5hero_id', '5is_pick'])

In [17]:
test1['1hero_id'] = hero_1
test1['1is_pick'] = heropick_1
test1['2hero_id'] = hero_2
test1['2is_pick'] = heropick_2
test1['3hero_id'] = hero_3
test1['3is_pick'] = heropick_3
test1['4hero_id'] = hero_4
test1['4is_pick'] = heropick_4
test1['5hero_id'] = hero_5
test1['5is_pick'] = heropick_5

In [18]:
X = test1.as_matrix()
X

/home/sulabh/software/anaconda3/envs/dotaenv36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[ 85,   0,  68, ...,   0,  73,   1],
       [ 71,   0,  73, ...,   0,   8,   1],
       [ 12,   0,  42, ...,   0,  85,   1],
       ...,
       [ 54,   0,  71, ...,   0,   7,   1],
       [ 44,   0,  29, ...,   0,  11,   1],
       [ 71,   0, 100, ...,   0,  39,   1]])

In [19]:
X.shape

(227312, 10)

In [20]:
y = np.array(hero_6)

In [21]:
y.shape

(227312,)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
from sklearn.neural_network import MLPClassifier

In [24]:
model1 = MLPClassifier()

In [25]:
model1.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [26]:
y_pred = model1.predict(X_test)

In [27]:
accuracy_score(y_test, y_pred)

0.10254492664364429

In [28]:
joblib.dump(model1, 'model1.pkl')

['model1.pkl']

### Model 2

In [30]:
test2 = pd.DataFrame(columns=[
                        '1hero_id',
                        '1is_pick',
                        '2hero_id',
                        '2is_pick', 
                        '3hero_id',
                        '3is_pick', 
                        '4hero_id', 
                        '4is_pick', 
                        '5hero_id', 
                        '5is_pick',
                        '6hero_id',
                        '6is_pick'])
test1['1hero_id'] = hero_1
test1['1is_pick'] = heropick_1
test1['2hero_id'] = hero_2
test1['2is_pick'] = heropick_2
test1['3hero_id'] = hero_3
test1['3is_pick'] = heropick_3
test1['4hero_id'] = hero_4
test1['4is_pick'] = heropick_4
test1['5hero_id'] = hero_5
test1['5is_pick'] = heropick_5
test1['6hero_id'] = hero_6
test1['6is_pick'] = heropick_6

X = test1.as_matrix()
y = np.array(hero_7)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model2 = MLPClassifier()
model2.fit(X_train, y_train)

/home/sulabh/software/anaconda3/envs/dotaenv36/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [31]:
y_pred = model2.predict(X_test)
accuracy_score(y_test, y_pred)
joblib.dump(model2, 'model2.pkl')

['model2.pkl']

### Model 3

In [33]:
test2 = pd.DataFrame(columns=[
                        '1hero_id',
                        '1is_pick',
                        '2hero_id',
                        '2is_pick', 
                        '3hero_id',
                        '3is_pick', 
                        '4hero_id', 
                        '4is_pick', 
                        '5hero_id', 
                        '5is_pick',
                        '6hero_id',
                        '6is_pick',
                        '7hero_id',
                        '7is_pick',
                        '8hero_id',
                        '8is_pick'])
test1['1hero_id'] = hero_1
test1['1is_pick'] = heropick_1
test1['2hero_id'] = hero_2
test1['2is_pick'] = heropick_2
test1['3hero_id'] = hero_3
test1['3is_pick'] = heropick_3
test1['4hero_id'] = hero_4
test1['4is_pick'] = heropick_4
test1['5hero_id'] = hero_5
test1['5is_pick'] = heropick_5
test1['6hero_id'] = hero_6
test1['6is_pick'] = heropick_6
test1['7hero_id'] = hero_7
test1['7is_pick'] = heropick_7
test1['8hero_id'] = hero_8
test1['8is_pick'] = heropick_8

X = test1.as_matrix()
y = np.array(hero_9)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model3 = MLPClassifier()
model3.fit(X_train, y_train)

/home/sulabh/software/anaconda3/envs/dotaenv36/lib/python3.6/site-packages/ipykernel_launcher.py:35: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [34]:
y_pred = model3.predict(X_test)
accuracy_score(y_test, y_pred)
joblib.dump(model3, 'model3.pkl')

['model3.pkl']

### Model 4

In [36]:
test2 = pd.DataFrame(columns=[
                        '1hero_id',
                        '1is_pick',
                        '2hero_id',
                        '2is_pick', 
                        '3hero_id',
                        '3is_pick', 
                        '4hero_id', 
                        '4is_pick', 
                        '5hero_id', 
                        '5is_pick',
                        '6hero_id',
                        '6is_pick',
                        '7hero_id',
                        '7is_pick',
                        '8hero_id',
                        '8is_pick',
                        '9hero_id',
                        '9is_pick',
                        '10hero_id',
                        '10is_pick'])
test1['1hero_id'] = hero_1
test1['1is_pick'] = heropick_1
test1['2hero_id'] = hero_2
test1['2is_pick'] = heropick_2
test1['3hero_id'] = hero_3
test1['3is_pick'] = heropick_3
test1['4hero_id'] = hero_4
test1['4is_pick'] = heropick_4
test1['5hero_id'] = hero_5
test1['5is_pick'] = heropick_5
test1['6hero_id'] = hero_6
test1['6is_pick'] = heropick_6
test1['7hero_id'] = hero_7
test1['7is_pick'] = heropick_7
test1['8hero_id'] = hero_8
test1['8is_pick'] = heropick_8
test1['9hero_id'] = hero_9
test1['9is_pick'] = heropick_9
test1['10hero_id'] = hero_10
test1['10is_pick'] = heropick_10

X = test1.as_matrix()
y = np.array(hero_11)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model4 = MLPClassifier()
model4.fit(X_train, y_train)

/home/sulabh/software/anaconda3/envs/dotaenv36/lib/python3.6/site-packages/ipykernel_launcher.py:43: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [37]:
y_pred = model4.predict(X_test)
accuracy_score(y_test, y_pred)
joblib.dump(model4, 'model4.pkl')

['model4.pkl']

### Model 5

In [38]:
test2 = pd.DataFrame(columns=[
                        '1hero_id',
                        '1is_pick',
                        '2hero_id',
                        '2is_pick', 
                        '3hero_id',
                        '3is_pick', 
                        '4hero_id', 
                        '4is_pick', 
                        '5hero_id', 
                        '5is_pick',
                        '6hero_id',
                        '6is_pick',
                        '7hero_id',
                        '7is_pick',
                        '8hero_id',
                        '8is_pick',
                        '9hero_id',
                        '9is_pick',
                        '10hero_id',
                        '10is_pick', 
                        '11hero_id', 
                        '11is_pick',
                        '12hero_id',
                        '12is_pick'])
test1['1hero_id'] = hero_1
test1['1is_pick'] = heropick_1
test1['2hero_id'] = hero_2
test1['2is_pick'] = heropick_2
test1['3hero_id'] = hero_3
test1['3is_pick'] = heropick_3
test1['4hero_id'] = hero_4
test1['4is_pick'] = heropick_4
test1['5hero_id'] = hero_5
test1['5is_pick'] = heropick_5
test1['6hero_id'] = hero_6
test1['6is_pick'] = heropick_6
test1['7hero_id'] = hero_7
test1['7is_pick'] = heropick_7
test1['8hero_id'] = hero_8
test1['8is_pick'] = heropick_8
test1['9hero_id'] = hero_9
test1['9is_pick'] = heropick_9
test1['10hero_id'] = hero_10
test1['10is_pick'] = heropick_10
test1['11hero_id'] = hero_11
test1['11is_pick'] = heropick_11
test1['12hero_id'] = hero_12
test1['12is_pick'] = heropick_12

X = test1.as_matrix()
y = np.array(hero_13)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model5 = MLPClassifier()
model5.fit(X_train, y_train)

/home/sulabh/software/anaconda3/envs/dotaenv36/lib/python3.6/site-packages/ipykernel_launcher.py:51: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [39]:
y_pred = model5.predict(X_test)
accuracy_score(y_test, y_pred)
joblib.dump(model5, 'model5.pkl')

['model5.pkl']

### Model 6

In [40]:
test2 = pd.DataFrame(columns=[
                        '1hero_id',
                        '1is_pick',
                        '2hero_id',
                        '2is_pick', 
                        '3hero_id',
                        '3is_pick', 
                        '4hero_id', 
                        '4is_pick', 
                        '5hero_id', 
                        '5is_pick',
                        '6hero_id',
                        '6is_pick',
                        '7hero_id',
                        '7is_pick',
                        '8hero_id',
                        '8is_pick',
                        '9hero_id',
                        '9is_pick',
                        '10hero_id',
                        '10is_pick', 
                        '11hero_id', 
                        '11is_pick',
                        '12hero_id',
                        '12is_pick',
                        '13hero_id', 
                        '13is_pick',
                        '14hero_id',
                        '14is_pick'])
test1['1hero_id'] = hero_1
test1['1is_pick'] = heropick_1
test1['2hero_id'] = hero_2
test1['2is_pick'] = heropick_2
test1['3hero_id'] = hero_3
test1['3is_pick'] = heropick_3
test1['4hero_id'] = hero_4
test1['4is_pick'] = heropick_4
test1['5hero_id'] = hero_5
test1['5is_pick'] = heropick_5
test1['6hero_id'] = hero_6
test1['6is_pick'] = heropick_6
test1['7hero_id'] = hero_7
test1['7is_pick'] = heropick_7
test1['8hero_id'] = hero_8
test1['8is_pick'] = heropick_8
test1['9hero_id'] = hero_9
test1['9is_pick'] = heropick_9
test1['10hero_id'] = hero_10
test1['10is_pick'] = heropick_10
test1['11hero_id'] = hero_11
test1['11is_pick'] = heropick_11
test1['12hero_id'] = hero_12
test1['12is_pick'] = heropick_12
test1['13hero_id'] = hero_13
test1['13is_pick'] = heropick_13
test1['14hero_id'] = hero_14
test1['14is_pick'] = heropick_14

X = test1.as_matrix()
y = np.array(hero_15)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model6 = MLPClassifier()
model6.fit(X_train, y_train)

/home/sulabh/software/anaconda3/envs/dotaenv36/lib/python3.6/site-packages/ipykernel_launcher.py:59: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [41]:
y_pred = model6.predict(X_test)
accuracy_score(y_test, y_pred)
joblib.dump(model6, 'model6.pkl')

['model6.pkl']

### Model 7

In [43]:
test2 = pd.DataFrame(columns=[
                        '1hero_id',
                        '1is_pick',
                        '2hero_id',
                        '2is_pick', 
                        '3hero_id',
                        '3is_pick', 
                        '4hero_id', 
                        '4is_pick', 
                        '5hero_id', 
                        '5is_pick',
                        '6hero_id',
                        '6is_pick',
                        '7hero_id',
                        '7is_pick',
                        '8hero_id',
                        '8is_pick',
                        '9hero_id',
                        '9is_pick',
                        '10hero_id',
                        '10is_pick', 
                        '11hero_id', 
                        '11is_pick',
                        '12hero_id',
                        '12is_pick',
                        '13hero_id', 
                        '13is_pick',
                        '14hero_id',
                        '14is_pick',
                        '15hero_id', 
                        '15is_pick',
                        '16hero_id',
                        '16is_pick'])
test1['1hero_id'] = hero_1
test1['1is_pick'] = heropick_1
test1['2hero_id'] = hero_2
test1['2is_pick'] = heropick_2
test1['3hero_id'] = hero_3
test1['3is_pick'] = heropick_3
test1['4hero_id'] = hero_4
test1['4is_pick'] = heropick_4
test1['5hero_id'] = hero_5
test1['5is_pick'] = heropick_5
test1['6hero_id'] = hero_6
test1['6is_pick'] = heropick_6
test1['7hero_id'] = hero_7
test1['7is_pick'] = heropick_7
test1['8hero_id'] = hero_8
test1['8is_pick'] = heropick_8
test1['9hero_id'] = hero_9
test1['9is_pick'] = heropick_9
test1['10hero_id'] = hero_10
test1['10is_pick'] = heropick_10
test1['11hero_id'] = hero_11
test1['11is_pick'] = heropick_11
test1['12hero_id'] = hero_12
test1['12is_pick'] = heropick_12
test1['13hero_id'] = hero_13
test1['13is_pick'] = heropick_13
test1['14hero_id'] = hero_14
test1['14is_pick'] = heropick_14
test1['15hero_id'] = hero_15
test1['15is_pick'] = heropick_15
test1['16hero_id'] = hero_16
test1['16is_pick'] = heropick_16

X = test1.as_matrix()
y = np.array(hero_17)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model7 = MLPClassifier()
model7.fit(X_train, y_train)

/home/sulabh/software/anaconda3/envs/dotaenv36/lib/python3.6/site-packages/ipykernel_launcher.py:67: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [44]:
y_pred = model7.predict(X_test)
accuracy_score(y_test, y_pred)
joblib.dump(model7, 'model7.pkl')

['model7.pkl']

### Model 8

In [46]:
test2 = pd.DataFrame(columns=[
                        '1hero_id',
                        '1is_pick',
                        '2hero_id',
                        '2is_pick', 
                        '3hero_id',
                        '3is_pick', 
                        '4hero_id', 
                        '4is_pick', 
                        '5hero_id', 
                        '5is_pick',
                        '6hero_id',
                        '6is_pick',
                        '7hero_id',
                        '7is_pick',
                        '8hero_id',
                        '8is_pick',
                        '9hero_id',
                        '9is_pick',
                        '10hero_id',
                        '10is_pick', 
                        '11hero_id', 
                        '11is_pick',
                        '12hero_id',
                        '12is_pick',
                        '13hero_id', 
                        '13is_pick',
                        '14hero_id',
                        '14is_pick',
                        '15hero_id', 
                        '15is_pick',
                        '16hero_id',
                        '16is_pick',
                        '17hero_id',
                        '17is_pick',
                        '18hero_id', 
                        '18is_pick',
                        '19hero_id',
                        '19is_pick'])
test1['1hero_id'] = hero_1
test1['1is_pick'] = heropick_1
test1['2hero_id'] = hero_2
test1['2is_pick'] = heropick_2
test1['3hero_id'] = hero_3
test1['3is_pick'] = heropick_3
test1['4hero_id'] = hero_4
test1['4is_pick'] = heropick_4
test1['5hero_id'] = hero_5
test1['5is_pick'] = heropick_5
test1['6hero_id'] = hero_6
test1['6is_pick'] = heropick_6
test1['7hero_id'] = hero_7
test1['7is_pick'] = heropick_7
test1['8hero_id'] = hero_8
test1['8is_pick'] = heropick_8
test1['9hero_id'] = hero_9
test1['9is_pick'] = heropick_9
test1['10hero_id'] = hero_10
test1['10is_pick'] = heropick_10
test1['11hero_id'] = hero_11
test1['11is_pick'] = heropick_11
test1['12hero_id'] = hero_12
test1['12is_pick'] = heropick_12
test1['13hero_id'] = hero_13
test1['13is_pick'] = heropick_13
test1['14hero_id'] = hero_14
test1['14is_pick'] = heropick_14
test1['15hero_id'] = hero_15
test1['15is_pick'] = heropick_15
test1['16hero_id'] = hero_16
test1['16is_pick'] = heropick_16
test1['17hero_id'] = hero_17
test1['17is_pick'] = heropick_17
test1['18hero_id'] = hero_18
test1['18is_pick'] = heropick_18
test1['19hero_id'] = hero_19
test1['19is_pick'] = heropick_19

X = test1.as_matrix()
y = np.array(hero_20)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model8 = MLPClassifier()
model8.fit(X_train, y_train)

/home/sulabh/software/anaconda3/envs/dotaenv36/lib/python3.6/site-packages/ipykernel_launcher.py:79: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [47]:
y_pred = model8.predict(X_test)
accuracy_score(y_test, y_pred)
joblib.dump(model8, 'model8.pkl')

['model8.pkl']

In [48]:
accuracy_score(y_test, y_pred)

0.03066229681279282